In [806]:
import pandas as pd
import numpy as np
from datetime import datetime

In [807]:
df = pd.read_excel('data/Bolsonaro + Lula - Validado.xlsx')

In [808]:
def day_for_flaot(x):
  if(isinstance(x, datetime)):
    x = float(f"{x.day}.{x.month}")
  return x

In [809]:
df['Duracao'] = df['Duracao'].apply(day_for_flaot)
df['Duracao'] = pd.to_numeric(df['Duracao'], errors='coerce', downcast='signed')

In [810]:
df['Duracao']

0      57.93
1      17.30
2      17.13
3      52.54
4      34.48
       ...  
564    51.34
565    15.20
566    85.10
567    12.93
568    49.19
Name: Duracao, Length: 569, dtype: float64

# Ao executar tudo será criado dois arquivos de dataset com os nomes 'file'_dataset.csv 'file'_dataset.xlsx

In [811]:
file = 'Amazon'
#file = 'Google'

In [812]:
if file.title() == 'Google':
  data = pd.read_csv('data/GoogleVision.csv')
  text = data.loc[data['Subclass']=='text'][['ID', 'Class']]
  text.rename({'Class': 'Text_Post'}, inplace=True, axis=1)
  data = data.loc[data['Subclass'] != 'text']
else:
  data = pd.read_csv('data/result_not_duplicates_oficial.csv')
  data.rename({'porcentagem': 'Percent', 'Nome': 'Class'}, axis=1, inplace=True)
  text = pd.DataFrame({'ID': [], 'Text_Post': []})

# Filtro de dados validados

In [813]:
data = data.loc[data['ID'].isin(df['ID'])]
text = text.loc[text['ID'].isin(df['ID'])]
data['ID'].unique().size

569

# Filtro de quartil

In [814]:
filter_percent = data['Percent'].quantile(.25)
filter_percent

57.73067474365234

In [815]:
print(f'''Quantidade Total: {data["ID"].count()}
      \nQuantidade esperado: {data["ID"].count()*.75}''')
data = data[data['Percent'] >= filter_percent]
print(f"Quantidade retornado: {data['ID'].count()}")

Quantidade Total: 11913
      
Quantidade esperado: 8934.75
Quantidade retornado: 8935


In [816]:
data[data['ID'] == 7150280209315417349]

,ID,Percent,Class
0,7150280209315417349,99.994194,Head
1,7150280209315417349,99.994194,Person
2,7150280209315417349,99.990639,Face
3,7150280209315417349,99.018929,Adult
4,7150280209315417349,99.018929,Male
5,7150280209315417349,99.018929,Man
6,7150280209315417349,92.266602,Photography
7,7150280209315417349,92.266602,Portrait


# DataFrame com ID, Class e o texto do Post

In [817]:
new_data = data.groupby('ID')['Class'].apply(' '.join).reset_index()
new_data = new_data.merge(text, how='outer', on='ID')
new_data

,ID,Class,Text_Post
0,7114971700365692165,Electrical Device Microphone People Person Cro...,NaN
1,7115033431473474822,Head Person Face Adult Male Man Photography Po...,NaN
2,7115050482179050758,Baseball Cap Cap Clothing Hat Head Person Adve...,NaN
3,7115120078982630661,Advertisement Poster Adult Male Man Person Peo...,NaN
4,7115161088219565317,People Person Crowd Accessories Formal Wear Ti...,NaN
...,...,...,...
564,7160017587160485125,People Person Adult Male Man Moped Motor Scoot...,NaN
565,7160037888611126534,Adult Male Man Person Face Head Flag Brazil Fl...,NaN
566,7160041692911930630,Crowd Person People Adult Male Man Electrical ...,NaN
567,7160077729751207174,Photography People Person Face Head Portrait B...,NaN


In [818]:
df.columns

Index(['Unnamed: 0', 'DataColeta', 'Perfil', 'DataPost', 'DiaDaSemana',
       'Plays', 'Curtidas', 'Comentarios', 'Compart.', 'Texto', 'LinkFoto',
       'LinkVideo', 'LinkPost', 'ID', 'Duracao', 'Duração Classificada',
       'Retórica Aristotélica', 'Dispositivo Retórico', 'Tipo de conteúdo',
       'Abordagem', 'Tonalidade', 'Main character', 'Texto / Hashtag'],
      dtype='object')

# Existem algumas colunas que podem vazar informação tome cuidado com elas

In [819]:
columns = [
  'ID',
  'Perfil',
  #'DataColeta',
  #'DataPost',
  #'LinkFoto',
  #'LinkVideo',             Links não são pertinentes
  #'LinkPost',
  'Duracao',
  #'Duração Classificada',
  #'Retórica Aristotélica',
  #'Dispositivo Retórico',
  #'Tipo de conteúdo',      Classificações manual
  #'Abordagem',
  #'Tonalidade',
  #'Main character',
  #'Texto / Hashtag',
  'DiaDaSemana',           #
  'Plays',                 #
  'Comentarios',           # Colunas que podem causar vazamento das informações
  'Compart.',              #
  'Texto',                 #
  'Curtidas',
]

In [820]:
df = df[columns]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           569 non-null    int64  
 1   Perfil       569 non-null    object 
 2   Duracao      569 non-null    float64
 3   DiaDaSemana  569 non-null    object 
 4   Plays        569 non-null    int64  
 5   Comentarios  569 non-null    int64  
 6   Compart.     569 non-null    int64  
 7   Texto        569 non-null    object 
 8   Curtidas     569 non-null    int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 40.1+ KB


## OBS na coluna perfil os valores repesentam
- 0 == bolsonaromessiasjair
- 1 == lulaoficial 

In [821]:
df = pd.get_dummies(df, columns=['Perfil'], prefix='', prefix_sep='', drop_first=True, dtype='int')
df.rename({'lulaoficial': 'Perfil'}, inplace=True, axis=1)

# Classifique de acordo com o quartil de cada variável de engajamento
# Segue a ordem conforme a importância
* ### 1 - Viral
* ### 2 - Bombou
* ### 3 - Mediano
* ###  4 - Flopou

In [822]:
def create_class_quartis(label: str):
  values = [
    df[label].quantile(.25),
    df[label].quantile(.5),
    df[label].quantile(.75),
    ]
  def create_class(x):
    if x < values[0]:
      return f"Flopou"
    if values[0] <= x < values[1]:
      return f"Mediano"
    if values[1] <= x < values[2]:
      return f"Bombou"
    if values[2] <= x:
      return f"Viral"
  
  print(f'Classes para {label}')
  print(f"Classe 1: 0 <= x < {values[0]}")
  print(f"Classe 2: {values[0]} <= x < {values[1]}")
  print(f"Classe 3: {values[1]} <= x < {values[2]}")
  print(f"Classe 4: {values[2]} <= x < ∞")
  return df[label].apply(create_class)
  

In [823]:
classes = ['Plays', 'Comentarios', 'Compart.', 'Curtidas',]
for label in classes:
  df['Classifier_'+label] = create_class_quartis(label)

Classes para Plays
Classe 1: 0 <= x < 125200.0
Classe 2: 125200.0 <= x < 259900.0
Classe 3: 259900.0 <= x < 671700.0
Classe 4: 671700.0 <= x < ∞
Classes para Comentarios
Classe 1: 0 <= x < 387.0
Classe 2: 387.0 <= x < 869.0
Classe 3: 869.0 <= x < 2547.0
Classe 4: 2547.0 <= x < ∞
Classes para Compart.
Classe 1: 0 <= x < 408.0
Classe 2: 408.0 <= x < 1166.0
Classe 3: 1166.0 <= x < 4048.0
Classe 4: 4048.0 <= x < ∞
Classes para Curtidas
Classe 1: 0 <= x < 10500.0
Classe 2: 10500.0 <= x < 24500.0
Classe 3: 24500.0 <= x < 72200.0
Classe 4: 72200.0 <= x < ∞


In [824]:
new_data = new_data.merge(df, on='ID', how='outer')

In [825]:
new_data.to_csv(f'data/{file}_dataset.csv')
new_data.to_excel(f'data/{file}_dataset.xlsx')